In [14]:

import ollama
import json
import numpy as np
import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Prepare Documents
with open("laws_json\\All_laws.json", "r", encoding="utf-8") as file:
    laws = json.load(file)
documents = [str(l[list(l.keys())[0]]) for l in laws]
print(documents[0:5])

# Save documents to a text file (for demonstration)
with open("data.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(documents))

# 2. Document Processing: Split the text into chunks
def chunk_documents():
    with open("data.txt", encoding="utf-8") as f:
        text = f.read()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50,
        length_function=len
    )
    return text_splitter.split_text(text)

chunks = chunk_documents()

# 3. Compute Embeddings for Each Chunk Using Ollama
# You may want to do this in batches if you have many chunks.
all_embeddings = []
for chunk in chunks:
    response = ollama.embeddings(
        model="snowflake-arctic-embed2",
        prompt=chunk
    )
    embedding = response["embedding"]
    # Optionally, check that embedding is non-empty.
    if not embedding:
        print("Empty embedding for chunk:", chunk[:100])
        continue
    all_embeddings.append(embedding)

# Convert the list of embeddings to a numpy array with type float32.
embeddings_np = np.array(all_embeddings, dtype=np.float32)

# Determine embedding dimension (assumes all embeddings have same dimension)
embedding_dim = embeddings_np.shape[1]
print(f"Embedding dimension: {embedding_dim}")

# 4. Build a Faiss Index
# Here we use a simple IndexFlatL2 for L2 distance (you can choose other indexes for larger datasets)
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings_np)
print(f"Faiss index size: {index.ntotal}")

# 5. Mapping: Keep track of the chunks corresponding to the embeddings.
# (Assuming no filtering happened; otherwise, adjust the mapping accordingly.)
indexed_chunks = chunks[:len(all_embeddings)]

# 6. Querying Function Using Faiss


['- Le code des impôts indirects comporte l’ensemble des dispositions légales relatives aux produitsperçus au profit de l’Etat ou des collectivités locales et compris sous le terme générique d’impôts indirects.', 'Outre la taxe sur la valeur ajoutée :− les alcools, les vins et autres boissons assimilées supportent, un droit de circulation ;− les ouvrages d’or, d’argent et de platine supportent un droit de garantie.Ces droits sont perçus au profit du budget de l’Etat, selon les règles fixées par le présent code.Les ouvrages d’or d’argent et de platine supportent, au profit du budget de l’Etat et selon les règles fixées parle présent code, un droit spécifique unique intitulé «droit de garantie».1', '- Les règles posées par les articles 4 à 46 qui suivent sont d’application générale. Toutefois, desdispositions spéciales à chacun des produits peuvent les compléter ou y déroger.2', 'Les personnes fabriquant les produits imposables ainsi que celles en faisant le commerce et, qui dansle prése

In [ ]:
def rag_query(query: str, temperature: float = 0.7):
    # Compute embedding for the query
    query_embedding = ollama.embeddings(
        model="snowflake-arctic-embed2",
        prompt=query
    )["embedding"]
    query_vector = np.array([query_embedding], dtype=np.float32)
    
    # Search the Faiss index for the 3 nearest neighbors
    k = 3
    distances, indices = index.search(query_vector, k)
    
    # Retrieve corresponding document chunks using the indices
    context_chunks = [indexed_chunks[i] for i in indices[0]]
    context = "\n\n".join(context_chunks)
    print("---------------Context---------------")
    print(context)
    
    # Generate answer using Ollama with retrieved context
    response = ollama.generate(
        model="mistral:latest",
        prompt=f"""Answer the question using only this context:
        {context}

        Question: {query}
        Answer:"""
    )
    
    return response["response"]

query = "c'est combien les taxes relative a l'achat des avions"
print("Question:", query)
print("Answer:", rag_query(query))


Question: c'est combien les taxes relative a l'achat des avions
---------------Context---------------
redevable doit déférer à plusieurs avis à tiers détenteursémanant respectivement de comptables chargés du recouvrement des impôts directs et des taxes sur lechiffre d‘affaires, il doit, en cas d‘insuffisance de ces deniers, exécuter les avis en proportion de leursmontants respectifs.Les

Lorsque les ventes et livraisons de biens meubles d‘occasions et assimilés sont effectuéespar un assujetti revendeur, la taxe est liquidée sur la marge déterminée par la différence entre le prixde vente TTC et le prix d‘achat TTC à condition que lesdits biens soient acquis auprès :- des

- Dans les abattoirs où le pesage des animaux avant l’abattage est seul possible, le poids net devant servir debase au calcul de la taxe est déterminé en appliquant au poids vif les pourcentages suivants :− 50 %, pour les ovidés, équidés et camelins ;− 50 %, pour les bœufs et taureaux ;− 55 %, pour
Answer:  The provide